In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.5 MB/s eta 0:00:00


In [ ]:
from transformers import GPT2LMHeadModel,PreTrainedTokenizerFast,AutoTokenizer,AutoModelWithLMHead,AdamW,get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import tqdm
import numpy as np
import pandas as pd
import os
import argparse

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class LyricsDataset(Dataset):
    def __init__(self, tokenizer, file_path):
        data = pd.read_csv(file_path, encoding='cp949')
        concats = [
            label + "&" + text for label, text in zip(data["genre"], data["lyrics"])
        ]
        self.item = tokenizer(
            concats,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=230,
        )["input_ids"]
        self.length = len(concats)

    def __getitem__(self, i):
        return self.item[i]

    def __len__(self):
        return self.length


def LyricsDataLoader(tokenizer, file_path, batch_size):
    data = LyricsDataset(tokenizer, file_path)
    return DataLoader(data, batch_size=batch_size)

In [ ]:
model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",unk_token='<unk>',sep_token='<unused0>',pad_token='<pad>',mask_token='<mask>')

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
import os
import argparse
import torch
import utils
from tqdm import tqdm
from transformers import (
    GPT2LMHeadModel,
    AutoTokenizer,
    AdamW,
    get_linear_schedule_with_warmup,
)


model_name = "skt/kogpt2-base-v2"
data_dir = '/content/drive/MyDrive/metamong/merged_lyrics.csv'
data = pd.read_csv(data_dir,encoding='cp949')
batch_size = 32
epochs = 10
lr = 2e-5
warmup_steps = 200


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({"pad_token": "<pad>","sep_token":"<unused0>","mask_token":"<mask>","unk_token":"<unk>"})
train_dataloader = LyricsDataLoader(
    tokenizer, data, batch_size
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
model.train()

optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps = warmup_steps, num_training_steps = -1
)

min_loss = 1e-9


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


TypeError: ignored

In [ ]:
for epoch in range(epochs):
  print(f"Training epoch {epoch}")
  for input_text in tqdm(train_dataloader):
    input_tensor = input_text.to(device)
    outputs = model(input_tensor, labels=input_tensor)
    loss = outputs[0]

    optimizer.zero_grad()
    model.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

  print(f"epoch {epoch} loss {outputs[0].item():0.2f}")

  labels = ['트로트','댄스','락','발라드']
  for label in labels:
    res = generate(label, tokenizer, model, 1)
    print(f"{label} : {res[0]}")

  if outputs[0].item() < min_loss:
    min_loss = outputs[0].item()
    model.save_pretrained("./save_model")

print("Done!")